- Fragmentation (from [here](https://dl.acm.org/citation.cfm?id=2893731))
- $\text{fragmentation}(x) = 1 - \frac{x \times\#(\text{free chuncks of size } x)}{\text{free space}}$

In [124]:
import random

In [120]:
def add_var(alloc_size=30, alloc_time=1, dealloc_time=10):
    return {"alloc_size":alloc_size, "alloc_time":alloc_time, "dealloc_time":dealloc_time}

In [121]:

mpattern = []

mpattern.append(add_var(1,2,3))
mpattern.append(add_var(1,2,3))
mpattern.append(add_var(1,2,3))

In [122]:
mpattern

[{'alloc_size': 1, 'alloc_time': 2, 'dealloc_time': 3},
 {'alloc_size': 1, 'alloc_time': 2, 'dealloc_time': 3},
 {'alloc_size': 1, 'alloc_time': 2, 'dealloc_time': 3}]

In [152]:
class mpattern:
    'Allocation pattern'
    allocation_list = [] # allocation_list is a list of byte sizes of objects to be allocated.
    size = 0 # memory size in bytes
    
    def __init__(self, size):
        self.allocation_list = []
        self.size = size
        self.free_mem = size
        self.memory = [ int(0) ] * size
        
    def add_var(self, alloc_size=30, alloc_time=0, dealloc_time=1):
        self.allocation_list.append({"alloc_size":alloc_size, "alloc_time":alloc_time, "dealloc_time":dealloc_time})
        self.free_mem -= alloc_size
        
    def allocate(self, size):
        allocated = False
        while not allocated:
            pick_place_to_allocate = random.randint(0,self.size-1-size)
            if self.isThereSpace(pick_place_to_allocate, size):
                self.writeAllocation(pick_place_to_allocate, size)
                
    def writeAllocation(self, position, size):
        self.memory[position: position+size] = [1] * size
            
    def isThereSpace(self, position, size):
        return sum(self.memory[position: position+size]) == 0
            
    def __str__(self):
        str1 = "size: {}\n".format(str(self.size))
        for var in self.allocation_list:
            #str1 += str(var) + "\n"
            str1 += "alloc_size: " + str(var["alloc_size"]) + "\n"
        str1 += str(self.memory)
        return str1
    
    def create_random(self, size, number):
        assert(False)
        
    def get_fragmentation(self, size_x):
        num_free_chunks_of_size_x = 0
        return size_x*num_free_chunks_of_size_x/self.free_mem

In [153]:
mem = mpattern(600)

In [154]:
mem.add_var(1)

In [155]:
mem.allocate(3)

KeyboardInterrupt: 

In [ ]:
print(mem)